### 1. Data Pre-processing

In [1]:
# Turning on the intellisense
%config IPCompleter.greedy=True

In [2]:
# Import the libraries
import os
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import optimizers, datasets, layers

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
# Suppress warnings and logs from GPU
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [4]:
# Prepare the mnist dataset
def mnist_dataset():
    (x, y), _ = datasets.mnist.load_data()
    ds = tf.data.Dataset.from_tensor_slices((x,y))
    ds = ds.map(prepare_mnist)
    ds = ds.take(20000).shuffle(20000).batch(100)
    return ds

@tf.function
def prepare_mnist(x,y):
    x = tf.cast(x, tf.float32)/255.0
    y = tf.cast(y, tf.int64)
    return x,y

In [5]:
# Building a Keras model
model = keras.Sequential([
    layers.Reshape(target_shape =(28 * 28 , ), input_shape = (28, 28)),
    layers.Dense(100, activation = 'relu'),
    layers.Dense(100, activation = 'relu'),
    layers.Dense(10)
])

optimizer = optimizers.Adam()

### 2. Building training model

In [6]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits = logits, labels = labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis = 1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)
    #computing gradients
    grads = tape.gradient(loss, model.trainable_variables)
    #update on weights
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

def train(epoch, model, optimizer):
    train_ds = mnist_dataset()
    loss = 0.0
    accuracy = 0.0
    for step, (x,y) in enumerate(train_ds):
        loss, accuracy = train_one_step(model, optimizer, x, y)
        if step % 500 == 0:
            print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
    return loss, accuracy

### 3. Training the model

In [7]:
for epoch in range(20):
    loss, accuracy = train(epoch, model, optimizer)

print('Final epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.3269005 ; accuracy 0.07
epoch 1 : loss 0.32610756 ; accuracy 0.89
epoch 2 : loss 0.15586588 ; accuracy 0.95
epoch 3 : loss 0.119178 ; accuracy 0.97
epoch 4 : loss 0.080849096 ; accuracy 0.97
epoch 5 : loss 0.054936837 ; accuracy 0.98
epoch 6 : loss 0.032290515 ; accuracy 0.99
epoch 7 : loss 0.026106087 ; accuracy 1.0
epoch 8 : loss 0.053299565 ; accuracy 0.98
epoch 9 : loss 0.040338274 ; accuracy 0.99
epoch 10 : loss 0.016881635 ; accuracy 0.99
epoch 11 : loss 0.014933702 ; accuracy 1.0
epoch 12 : loss 0.0102891205 ; accuracy 1.0
epoch 13 : loss 0.0063590617 ; accuracy 1.0
epoch 14 : loss 0.0032029096 ; accuracy 1.0
epoch 15 : loss 0.050467744 ; accuracy 0.99
epoch 16 : loss 0.0043904707 ; accuracy 1.0
epoch 17 : loss 0.011554726 ; accuracy 0.99
epoch 18 : loss 0.0036839328 ; accuracy 1.0
epoch 19 : loss 0.0012076498 ; accuracy 1.0
Final epoch 19 : loss 0.0017735341 ; accuracy 1.0
